In [1]:
#sqlite

In [1]:
import sqlite3

In [2]:
connection = sqlite3.connect('test.db')

In [3]:
cursor = connection.cursor()

In [58]:
cursor.execute('update products_history set price_sale = 42616 where id = 2856')

In [60]:
cursor.execute('select id, price, price_sale, title, url from products_history where title like "%LUPO AX 120%" limit 1000')

In [269]:
cursor.execute('select datetime_create from products_history where id = 9 and  datetime_create >= "2022-11-04"')
cursor.fetchall()

[('2022-11-27 15:37:19.581614',)]

In [61]:
#cursor.execute('select * from products_history where title like "%боти%" limit 1000')

In [62]:
#cursor.execute('select * from products_history where id = 15 limit 10')

In [64]:
#cursor.fetchall()

q = """SELECT CalendarYear, CalendarQuarter, SalesAmountQuota AS SalesQuota,  
       LAG(SalesAmountQuota,1,0) OVER (ORDER BY CalendarYear, CalendarQuarter) AS PrevQuota,  
       SalesAmountQuota - LAG(SalesAmountQuota,1,0) OVER (ORDER BY CalendarYear, CalendarQuarter) AS Diff  
FROM dbo.FactSalesQuota  
WHERE EmployeeKey = 272 AND CalendarYear IN (2001, 2002)  
ORDER BY CalendarYear, CalendarQuarter;"""

In [119]:
q = """
SELECT 
    id, 
    title,
    price_sale,  
    LAG(price_sale,1,0) OVER (ORDER BY id) AS PrevQuota,  
    price_sale - LAG(price_sale,1,0) OVER (ORDER BY id desc) AS Diff  
FROM products_history
WHERE title like "%LUPO AX 120%" 
ORDER BY id desc
LIMIT 4
"""

In [134]:
q = """
SELECT 
    id, 
    title,
    price_sale,  
    LEAD(price_sale,1,0) OVER (ORDER BY id) AS PrevQuota,  
    price_sale - LEAD(price_sale,1,0) OVER (PARTITION BY title ORDER BY title, id desc) AS Diff  
FROM products_history
--WHERE title like "%LUPO AX 120%" 
where title like "%боти%"
ORDER BY title, id desc
--LIMIT 4
"""

In [242]:
q = """
SELECT 
    id, 
    title,
    price_sale,  
    LEAD(price_sale,1,0) OVER (ORDER BY title desc, id desc) AS PrevQuota,  
    price_sale - LEAD(price_sale,1,0) OVER ( ORDER BY title desc, id desc) AS Diff  
FROM products_history
--WHERE title like "%LUPO AX 120%" 
--where title like "%боти%"
where title like "%130%" and title like "%боти%"
--ORDER BY title, id desc
ORDER BY title desc, id desc
--LIMIT 4
"""

In [243]:
cursor.execute(q)

for i in cursor.fetchall():
    print(i)

q = """
SELECT 
    id, 
    title,
    price_sale,  
    LEAD(price_sale,1,0) OVER (ORDER BY title desc, id desc) AS PrevQuota,  
    price_sale - LEAD(price_sale,1,0) OVER (PARTITION BY title  ORDER BY title desc, id desc ) AS Diff,
    ROW_NUMBER() OVER(PARTITION BY title ORDER BY id desc) AS row_number

FROM products_history
--WHERE title like "%LUPO AX 120%" 
--where title like "%боти%"
where title like "%130%" and title like "%боти%"--and title like "%ALLTRACK ELITE 130%"
--ORDER BY title, id desc
--having row_number = 1
--ORDER BY row_number, title desc, id desc
ORDER BY title desc, id desc
--LIMIT 4
"""
cursor.execute(q)
for i in cursor.fetchall():
    print(i)

q = """
SELECT 
    id, 
    title,
    price_sale,  
    LEAD(price_sale,1,0) OVER (PARTITION BY title  ORDER BY title desc, id desc) AS PrevQuota,  
    price_sale - LEAD(price_sale,1,0) OVER (PARTITION BY title  ORDER BY title desc, id desc ) AS Diff,
    ROW_NUMBER() OVER(PARTITION BY title ORDER BY id desc) AS row_number

FROM products_history
--WHERE title like "%LUPO AX 120%" 
--where title like "%боти%"
where title like "%130%" and title like "%боти%"--and title like "%ALLTRACK ELITE 130%"
--ORDER BY title, id desc
--having row_number = 1
--ORDER BY row_number, title desc, id desc
ORDER BY title desc, id desc
--LIMIT 4
"""
cursor.execute(q)
for i in cursor.fetchall():
    print(i)

In [254]:
cursor.execute('update products_history set price_sale = 40047 where id = 97')

In [255]:
WITH sales_numbered AS (
  SELECT id, product_id, salesperson_id, amount,
    row_number() over(PARTITION BY product_id ORDER BY amount DESC) AS rn
  FROM sale
)
SELECT id, product_id, salesperson_id, amount
FROM sales_numbered
WHERE rn = 1;

SyntaxError: invalid syntax (4184171144.py, line 1)

In [262]:
q = """
WITH sales_numbered AS (
    SELECT 
        id, 
        title,
        price_sale,  
        LEAD(price_sale,1,0) OVER (PARTITION BY title ORDER BY  title, id desc ) AS prev_price_sale,  
        price_sale - LEAD(price_sale,1,0) OVER (PARTITION BY title  ORDER BY title, id desc ) AS diff,
        ROW_NUMBER() OVER(PARTITION BY title ORDER BY id desc) AS row_number

    FROM products_history
    --where datetime_create >= NOW() - INTERVAL 1 DAY
    ORDER BY id desc
    LIMIT 1000000
)
SELECT 
    id, 
    title,
    price_sale,  
    prev_price_sale,
    diff,
    row_number
FROM sales_numbered
WHERE row_number = 1 and diff !=0  ;

"""
cursor.execute(q)
for i in cursor.fetchall():
    print(i)

(2860, 'Горнолыжные ботинки Rossignol ALLTRACK ELITE 130 LT GW', 40047, 50047, -10000, 1)
(2856, 'Горнолыжные ботинки Dalbello LUPO AX 120', 42616, 44616, -2000, 1)
(2764, 'Павелецкая Сити 3 34 96.3 ул. Дубининская, д. 59-69 55.71962004653103,37.63727228350035', 42727373, 42685218, 42155, 1)
(2735, 'Павелецкая Сити 1 28 41.4 ул. Дубининская, д. 59-69 55.71962004653103,37.63727228350035', 23608514, 23158828, 449686, 1)
(2707, 'Павелецкая Сити 3 45 87.4 ул. Дубининская, д. 59-69 55.71962004653103,37.63727228350035', 39543717, 39505458, 38259, 1)
(2629, 'Причальный Север 25 68.0 Причальный пр., д. 8 55.76018,37.517482', 25462903, 0, 25462903, 1)
(2613, 'Павелецкая Сити 1 2 61.6 ул. Дубининская, д. 59-69 55.71962004653103,37.63727228350035', 25585714, 25323914, 261800, 1)
(2611, 'Причальный Север 23 68.0 Причальный пр., д. 8 55.76018,37.517482', 25273798, 0, 25273798, 1)
(2597, 'Причальный Север 10 35.0 Причальный пр., д. 8 55.76018,37.517482', 14484876, 0, 14484876, 1)
(2593, 'Причальный 

In [7]:
#достаем не приехавшие обект
#- выбрать все торы за два 
#- сгрупировать по уникальному полю
#- псочитать 
#- там где 1 элемент занчит он или новый пришедший вывоить пминимальную дату групирвоа
#- если дата вчера то занчит старый элемент не обновился

In [14]:
q = """
WITH sales_numbered AS (
    SELECT 
        id, 
        title,
        price_sale,  
        url,
        datetime_create,
        count(1) as c
    FROM products_history
    where datetime_create >= '2022-12-05'
    group by url, title
    having c = 1
    ORDER BY id desc
    LIMIT 1000000
)
SELECT 
    id, 
    title,
    price_sale,  
    url
FROM sales_numbered
where datetime_create <= '2022-12-06'
"""
cursor.execute(q)
for i in cursor.fetchall():
    print(i)

(1183, 'Причальный Север 17 37.3 Причальный пр., д. 8 55.76018,37.517482', 15781018, 'https://level.ru/prichal/flat/2room/s2-330/')
(1079, 'Причальный Юг 31 107.6 Причальный пр., д. 8 55.76018,37.517482', 45398112, 'https://level.ru/prichal/flat/4room/ju2-491/')
(1042, 'Южнопортовая 5 6 75.0 ул. Южнопортовая, вл.28-28а 55.705727,37.696445', 17802355, 'https://level.ru/southport/flat/3room/5-52/')
(826, 'Южнопортовая 4 8 76.5 ул. Южнопортовая, вл.28-28а 55.705727,37.696445', 16097329, 'https://level.ru/southport/flat/3room/4-66/')


q = """
WITH sales_numbered AS (
    SELECT 
        id, 
        title,
        price_sale,  
        url,
        datetime_create,
        count(1) as c
    FROM products_history
    where datetime_create >= '2022-12-05'
    group by url, title
    having c = 1
    ORDER BY id desc
    LIMIT 1000000
)
SELECT 
    id, 
    title,
    price_sale,  
    url
FROM sales_numbered
where datetime_create >= '2022-12-06'
"""
cursor.execute(q)
for i in cursor.fetchall():
    print(i)

q = """
SELECT 
    count(1) as c,
    url, title
FROM products_history
where datetime_create >= "2022-12-05"
group by url, title
ORDER BY id desc
LIMIT 1000000
"""
cursor.execute(q)
for i in cursor.fetchall():
    print(i)